In [1]:
import time
import pickle
import numpy as np
import pandas as pd
import urllib.parse
import requests
import platform
import matplotlib.pyplot as plt
import missingno as msno

from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from matplotlib import font_manager, rc


plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Linux':
    rc('font', family='NanumBarunGothic')
else:
    print('Unknown system... sorry~~')

In [401]:
cols = ['name', 'market_value', 'app', 'conceded_goals', 'clean_sheets',
        'yellow_card', 'second_yell', 'red_card', 'age', 'height', 'citizenship',
        'position', 'foot', 'agent', 'club', 'joined', 'expire', 'last_contract',
        'outfitter','sns','caps_goals']

In [392]:
df = pd.DataFrame(columns=cols)
df

,name,market_value,app,conceded_goals,clean_sheets,yellow_card,second_yell,red_card,age,height,...,position,foot,agent,club,joined,expire,last_contract,outfitter,sns,caps_goals


In [393]:
league, order = "premier-league", "GB1"
base_url = f"https://www.transfermarkt.co.uk/{league}/startseite/wettbewerb/{order}"

headers = {'User-Agent': UserAgent().chrome}
response = requests.get(base_url, headers=headers)
dom = BeautifulSoup(response.content, "html.parser")

# 리그별 클럽의 링크데이터
elements = dom.select("#yw1 > table > tbody > tr")

base_link = "https://www.transfermarkt.co.uk"
links = []
for element in elements:
    link = element.select_one("a").get("href")
    link = urllib.parse.urljoin(base=base_link, url=link, allow_fragments=True)
    links.append(link)

for link in links:
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    
    # 리그의 선수 링크데이터
    elements = dom.select("#yw1 > table > tbody > tr")

    player_links = []
    for element in elements:
        player_link = element.select('a')[-1].get('href')
        player_link = urllib.parse.urljoin(base=base_link, url=player_link, allow_fragments=True)
        player_links.append(player_link)
            
        # 선수별 정보데이터
    for player_link in player_links:
        headers = {'User-Agent': UserAgent().chrome}
        response = requests.get(player_link, headers=headers)
        dom = BeautifulSoup(response.content, 'html.parser')

        name = dom.select_one('#main > div:nth-child(12) > div > div > div.dataMain > div > div.dataName > h1').text
        print(name)
        try:
            vals = dom.select('#main > div:nth-child(12) > div > div > div.dataContent > div > div:nth-child(3) > p.hide-for-small.notTablet > span.dataValue > a')
            caps_goals = int(vals[0].text) / int(vals[1].text)
        except:
            caps_goals = None
        try:
            market_value = dom.select_one('#main > div:nth-child(17) > div.large-8.columns > div:nth-child(2) > div.row.collapse > div.large-6.large-push-6.small-12.columns > div > div.box.viewport-tracking > div.row.collapse > div > div > div > div.row.collapse > div.large-5.columns.small-12 > div > div.zeile-oben > div.right-td > a:nth-child(1)').text
        except:
            market_value = None
        try:
            app = dom.find_all('span', 'wert')[0].text
            conceded_goals = dom.find_all('span', 'wert')[2].text
            clean_sheets = dom.find_all('span', 'wert')[4].text
            yellow_card = dom.find_all('span', 'wert')[1].text
            second_yell = dom.find_all('span', 'wert')[3].text
            red_card = dom.find_all('span', 'wert')[5].text
        except:
            app, conceded_goals, clean_sheets, yellow_card, second_yell, red_card = None, None, None, None, None, None

        for info in dom.find_all('tr')[:15]:
            if info.find('th') is None:
                continue

            if info.find('th').text.strip()[:-1] == "Age":
                age = info.find('td').text
            elif info.find('th').text[:-1] == "Height":
                height = info.find('td').text
            elif info.find('th').text[:-1] == "Citizenship":
                citizenship = len(np.unique(info.find('td').text))
            elif info.find('th').text[:-1] == "Position":
                position = info.find('td').text.replace(" ", "").replace("\n", "")
            elif info.find('th').text[:-1] == "Foot":
                foot = info.find('td').text
            elif info.find('th').text[:-1] == "Player agent":
                agent = info.find('td').text.replace("\n", "")
            elif info.find('th').text.strip()[:-1] == "Current club":
                club = info.find('td').text.strip()
            elif info.find('th').text.strip()[:-1] == "Joined":
                joined = info.find('td').text.strip()
            elif info.find('th').text.strip()[:-1] == "Contract expires":
                expire = info.find('td').text.strip()
            elif info.find('th').text.strip()[:-1] == "Date of last contract extension":
                last_contract = info.find('td').text.strip()
            elif info.find('th').text.strip()[:-1] == "Outfitter":
                outfitter = info.find('td').text.strip()
            elif info.find('th').text.strip()[:-1] == "Social-Media":
                sns = len(info.find_all('a'))

        data = {
            'name': name, 'market_value': market_value, 'caps_goals': caps_goals, 'app': app,
            'conceded_goals': conceded_goals,
            'clean_sheets': clean_sheets, 'yellow_card': yellow_card, 'second_yell': second_yell,
            'red_card': red_card, 'age': age, 'height': height, 'citizenship': citizenship,
            'position': position, 'foot': foot, 'agent': agent, 'club': club, 'joined': joined,
            'expire': expire, 'last_contract': last_contract, 'outfitter': outfitter, 'sns': sns
        }

        df.loc[len(df)] = data
        time.sleep(0.5)

Ederson
Zack Steffen
Scott Carson
Rúben Dias
Aymeric Laporte
Nathan Aké
John Stones
Eric García
Philippe Sandler
Benjamin Mendy
Oleksandr Zinchenko
João Cancelo
Kyle Walker
Rodri
Fernandinho
Phil Foden
Ilkay Gündogan
Kevin De Bruyne
Bernardo Silva
Raheem Sterling
Ferran Torres
Riyad Mahrez
Gabriel Jesus
Sergio Agüero
Alisson
Caoimhin Kelleher
Adrián
Virgil van Dijk
Joe Gomez
Ozan Kabak
Joel Matip
Rhys Williams
Nathaniel Phillips
Ben Davies
Andrew Robertson
Konstantinos Tsimikas
Trent Alexander-Arnold
Neco Williams
Fabinho
Naby Keïta
Georginio Wijnaldum
Thiago
Jordan Henderson
Alex Oxlade-Chamberlain
James Milner
Curtis Jones
Ben Woodburn
Sadio Mané
Diogo Jota
Mohamed Salah
Xherdan Shaqiri
Roberto Firmino
Divock Origi
Edouard Mendy
Kepa
Willy Caballero
Kurt Zouma
Andreas Christensen
Antonio Rüdiger
Thiago Silva
Ben Chilwell
Emerson
Marcos Alonso
Reece James
César Azpilicueta
Jorginho
N'Golo Kanté
Mateo Kovacic
Billy Gilmour
Kai Havertz
Mason Mount
Charly Musonda Jr.
Callum Hudson-Odoi
C

In [396]:
with open('datas/premier_players.pkl', 'wb') as f:
    pickle.dump(df, f)

In [2]:
with open('datas/premier_players.pkl', 'rb') as f:
    datas = pickle.load(f)

In [3]:
datas.head()

,name,market_value,app,conceded_goals,clean_sheets,yellow_card,second_yell,red_card,age,height,...,position,foot,agent,club,joined,expire,last_contract,outfitter,sns,caps_goals
0,Ederson,£50.40m,32,23,17,3,-,-,27,"1,88 m",...,Goalkeeper,left,Gestifute,Manchester City,"Jul 1, 2017","Jun 30, 2025","May 13, 2018",Puma,3,NaN
1,Zack Steffen,£5.40m,1,1,-,-,-,-,26,"1,91 m",...,Goalkeeper,right,Wasserman,Manchester City,"Jul 9, 2019","Jun 30, 2023","May 13, 2018",Nike,2,NaN
2,Scott Carson,£450Th.,-,-,-,-,-,-,35,"1,88 m",...,Goalkeeper,right,Wasserman,Manchester City,"Aug 8, 2019","May 31, 2021","Aug 19, 2020",Nike,2,NaN
3,Rúben Dias,£63.00m,29,1,-,3,-,-,23,"1,87 m",...,Defender-Centre-Back,right,Gestifute,Manchester City,"Sep 29, 2020","Jun 30, 2026","Aug 19, 2020",Nike,2,13.5
4,Aymeric Laporte,£45.00m,14,-,-,1,-,-,26,"1,89 m",...,Defender-Centre-Back,left,Wasserman,Manchester City,"Jan 30, 2018","Jun 30, 2025","Feb 22, 2019",adidas,3,None


In [35]:
with open('datas/final_total_df.pkl', 'wb') as f:
    pickle.dump(final_dfs, f)

---

---

---